In [57]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim


# Wine Custom Dataset Class

In [40]:
class WineDatset(Dataset):
  def __init__(self, directory):
    wineDataset = pd.read_csv(directory)
    features=wineDataset.iloc[:, 0:11]
    labels=wineDataset.iloc[:, 11:12]
    
    self.features = features
    self.labels = labels

  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, idx):
    sample = {'Features': self.features.iloc[idx], 'Labels': self.labels.iloc[idx]}
    return sample

In [41]:
wineDataset = WineDatset('/content/winequality-red.csv')

In [42]:
test_pct = 0.2
test_size = int(len(wineDataset)*test_pct)
train_size = len(wineDataset) - test_size
train_ds, test_ds = torch.utils.data.random_split(wineDataset, [train_size, test_size])

#Dataloader

In [100]:
train_dl = DataLoader(train_ds, 5, shuffle=True, num_workers=2,collate_fn=lambda x: x,pin_memory=True)
test_dl = DataLoader(test_ds, 5, num_workers=2, collate_fn=lambda x: x,pin_memory=True)

In [101]:
for i_batch, sample_batched in enumerate(train_dl):
  # observe 1st batch and stop.
  if i_batch == 1:
      batch = [sample_batched[i]['Features'] for i in range(0,len(sample_batched))]
      print(batch)
      break

[fixed acidity             7.70000
volatile acidity          0.58000
citric acid               0.10000
residual sugar            1.80000
chlorides                 0.10200
free sulfur dioxide      28.00000
total sulfur dioxide    109.00000
density                   0.99565
pH                        3.08000
sulphates                 0.49000
alcohol                   9.80000
Name: 242, dtype: float64, fixed acidity            5.60000
volatile acidity         0.91500
citric acid              0.00000
residual sugar           2.10000
chlorides                0.04100
free sulfur dioxide     17.00000
total sulfur dioxide    78.00000
density                  0.99346
pH                       3.68000
sulphates                0.73000
alcohol                 11.40000
Name: 1178, dtype: float64, fixed acidity             8.5000
volatile acidity          0.2800
citric acid               0.5600
residual sugar            1.8000
chlorides                 0.0920
free sulfur dioxide      35.0000
total sul

In [55]:
len(test_dl)

64

#Neural Network Class with 3 FC Layers

In [102]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.bn = nn.BatchNorm1d(11)
        self.fc1 = nn.Linear(11, 20)
        self.fc2 = nn.Linear(20, 30)
        self.fc3 = nn.Linear(30, 1)

    def forward(self, x):
        x=self.fc1(x)
        x=F.relu(x)
        x=self.fc2(x)
        x=F.relu(x)
        x=self.fc3(x)
        return x

net = Net()

In [103]:
def mean_squared_error(actual, predicted):
	sum_square_error = 0.0
	for i in range(len(actual)):
		sum_square_error += (actual[i] - predicted[i])**2.0
	mean_square_error = 1.0 / len(actual) * sum_square_error
	return mean_square_error

In [104]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [105]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl):
        # get the inputs; data is a list of [inputs, labels]
        features, labels = [data[i]['Features'] for i in range(0,len(data))], [data[i]['Labels'] for i in range(0,len(data))]
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(torch.as_tensor(features).float())
        loss = mean_squared_error(outputs, torch.as_tensor(labels))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 4.592
[1,   200] loss: 0.368
[2,   100] loss: 0.320
[2,   200] loss: 0.306
[3,   100] loss: 0.362
[3,   200] loss: 0.285
[4,   100] loss: 0.383
[4,   200] loss: 0.273
[5,   100] loss: 0.315
[5,   200] loss: 0.346
[6,   100] loss: 0.315
[6,   200] loss: 0.316
[7,   100] loss: 0.317
[7,   200] loss: 0.320
[8,   100] loss: 0.349
[8,   200] loss: 0.308
[9,   100] loss: 0.328
[9,   200] loss: 0.329
[10,   100] loss: 0.311
[10,   200] loss: 0.328
[11,   100] loss: 0.310
[11,   200] loss: 0.340
[12,   100] loss: 0.324
[12,   200] loss: 0.314
[13,   100] loss: 0.335
[13,   200] loss: 0.306
[14,   100] loss: 0.328
[14,   200] loss: 0.315
[15,   100] loss: 0.367
[15,   200] loss: 0.285
[16,   100] loss: 0.315
[16,   200] loss: 0.329
[17,   100] loss: 0.312
[17,   200] loss: 0.344
[18,   100] loss: 0.349
[18,   200] loss: 0.311
[19,   100] loss: 0.359
[19,   200] loss: 0.320
[20,   100] loss: 0.308
[20,   200] loss: 0.323
[21,   100] loss: 0.320
[21,   200] loss: 0.320
[22,   100

In [107]:
PATH = 'wine-model.pth'
torch.save(net.state_dict(), PATH)

In [108]:
correct = 0
sum1 = 0
with torch.no_grad():
    for i, data in enumerate(test_dl):
        features, labels = [data[i]['Features'] for i in range(0,len(data))], [data[i]['Labels'] for i in range(0,len(data))]
        outputs = net(torch.as_tensor(features).float())
        _, predict = torch.max(outputs.data, 1)
        labels = torch.as_tensor(labels)
        sum1 += labels.size(0)
        correct += (predict == labels).sum().item()

print('Accuracy: %d %%' % (100 * correct / sum1))

Accuracy: 0 %
